+ 데이터가 너무 많은 관계로 지식그래프 생성 시 월별로 나누어 생성하도록 하였음.

In [1]:
import csv
import itertools
import pandas as pd
import networkx as nx
from itertools import product
import matplotlib.pyplot as plt

data = {
    
    'hashtags': [],
    'importance': [],
    'keyword': []
    
}

# 실제 CSV 파일 경로로 수정
csv_file_path = 'Market_7.csv'

with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 첫 번째 행은 열 제목이므로 건너뜁니다.

    for row in csv_reader:
        hashtags = row[1]
        importance = float(row[2])
        keywords = row[3]
        

        hashtags = [tag.strip("[]").replace("'", "").split(', ') for tag in hashtags.split(", ")]
        keywords = [keyword.strip("[]").replace("'", "").split(', ') for keyword in keywords.split(", ")]

        data['importance'].append([importance])
        data['hashtags'].append(hashtags)
        data['keyword'].append(keywords)

data

{'hashtags': [[['컬리카레우동'],
   ['마켓컬리카레우동'],
   ['카레우동'],
   ['카레우동밀키트'],
   ['냉동카레우동'],
   ['엄마밥상카레우동'],
   ['국물카레우동'],
   ['카레우동추천'],
   ['카레우동내돈내산'],
   ['내돈내산카레우동']],
  [['마켓컬리'],
   ['마켓컬리'],
   ['오늘도컬리'],
   ['ID_wlsdud6818'],
   ['마켓컬리밀키트'],
   ['마켓컬리솥밥'],
   ['솥밥밀키트'],
   ['마켓컬리밀키트추천'],
   ['밀키트추천'],
   ['공감'],
   ['컬리'],
   ['수린한우사골솥밥밀키트'],
   ['수린사골솥밥밀키트'],
   ['집들이메뉴'],
   ['집들이밀키트']],
  [['마켓컬리베테랑'],
   ['베테랑칼국수밀키트'],
   ['베테랑칼국수'],
   ['마켓컬리베테랑칼국수'],
   ['내돈내산리뷰'],
   ['베테랑칼국수리뷰'],
   ['맛집밀키트'],
   ['전주베테랑칼국수']],
  [['마켓컬리'],
   ['마켓컬리'],
   ['마켓컬리'],
   ['마켓컬리소불고기'],
   ['소불고기'],
   ['대용량소불고기'],
   ['마켓컬리반찬'],
   ['마켓컬리간편식'],
   ['마켓컬리밀키트'],
   ['마켓컬리불고기'],
   ['마켓컬리양념불고기']],
  [['복담'],
   ['단호박찹쌀떡'],
   ['찹쌀떡'],
   ['마켓컬리'],
   ['마켓컬리추천'],
   ['마켓컬리간식'],
   ['마켓컬리간식추천'],
   ['내돈내산'],
   ['컬리'],
   ['컬리추천'],
   ['컬리간식추천']],
  [['젝시믹스'],
   ['뮬라웨어'],
   ['내돈내산'],
   ['체크인챌린지'],
   ['제시믹스'],
   ['뮬라웨어'],
   ['내돈내산'],
   ['소비기록'],
   ['요가복추천'],
   ['뮬라웨어할인쿠폰'],
   ['제시믹스할인쿠폰'

In [2]:
def create_dataframe(data):
    all_combinations = []

    # 모든 importance 값에 대해 가능한 조합 생성
    for importance, hashtags, keyword in zip(data['importance'], data['hashtags'], data['keyword']):
        # importance를 리스트로 변환하여 사용
        importance_list = [importance]
        combinations = list(product(importance_list, hashtags, keyword))
        all_combinations.extend(combinations)

    # 데이터프레임 생성
    df = pd.DataFrame(all_combinations, columns=['weight', 'to', 'from'])

    return df
result_df=create_dataframe(data)
result_df.head()

,weight,to,from
0,[6.0],[컬리카레우동],[food]
1,[6.0],[컬리카레우동],[라이프스타일]
2,[6.0],[컬리카레우동],[컬리]
3,[6.0],[마켓컬리카레우동],[food]
4,[6.0],[마켓컬리카레우동],[라이프스타일]


In [3]:
import re

def del_list(text):
    # 리스트를 문자열로 변환 후 대괄호 제거
    text = re.sub(r'\[|\]', '', str(text))
    text=re.sub(r'\'', '',str(text))

    return text

result_df['from']= result_df['from'].apply(del_list)
result_df['to']= result_df['to'].apply(del_list)
result_df['weight']=result_df['weight'].apply(del_list)
result_df['weight']=result_df['weight'].astype('float')

In [4]:
result_df.head()


,weight,to,from
0,6.0,컬리카레우동,food
1,6.0,컬리카레우동,라이프스타일
2,6.0,컬리카레우동,컬리
3,6.0,마켓컬리카레우동,food
4,6.0,마켓컬리카레우동,라이프스타일


In [5]:

# 'to' 또는 'from' 열에서 '마켓컬리'를 포함하는 행 제거
filtered_df = result_df[~result_df['to'].str.contains('마켓컬리') & ~result_df['from'].str.contains('마켓컬리')]
filtered_df = result_df[~result_df['to'].str.contains('컬리') & ~result_df['from'].str.contains('컬리')]

A=filtered_df
A.to_csv('Market_7.csv',index=False)